# Group 55 Project Proposal: Type 2 Diabetes Based on Glucose and BMI 

### Introduction

**Relevant Background:** Type 2 diabetes is the result of one’s inability to properly use insulin when high levels of sugar are introduced to the body. Insulin is used to absorb glucose out of the blood system; however, if the body does not make any insulin, which is the case for patients with type 2 diabetes, the glucose levels may not be managed to a safe level. While type 2 diabetes is preventable, there are many factors that can induce the development of type 2 diabetes such as food intake, genetics, or weight.  

**Dataset Information:** The dataset we will be using includes the data for type 2 diabetic cases for female Pima Indian Heritage patients of at least 20 years of age. The dataset includes the number of pregnancies, glucose levels in the blood, blood pressure, the thickness of the skin, insulin levels in the blood, body mass index (BMI), diabetes pedigree function which is the likelihood of getting type 2 diabetes, age and outcome which is either 1 (has diabetes) or 0 (doesn’t have diabetes.) 

**Predictive Question:** What glucose and BMI values indicate type 2 diabetes? 

### Preliminary Exploratory Data Analysis

In [8]:
# importing libraries 
library(tidyverse)
library(repr)

In [10]:
# importing data

diabetes <- read_csv("https://raw.githubusercontent.com/kzhang1223/dsci-100-2023s1-group-55/main/diabetes.csv")

Rows: 768 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (9): Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, D...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
# table of data 

In [ ]:
# visualization

### Methods

We will use the k-nearest neighbors classifier to create a model that will predict whether one has or does not have diabetes based on glucose levels in the blood and BMI. Through this model, we will use a 5-fold cross-validation to determine the best k value for the highest accuracy. This classifier would require high accuracy as diabetes should be treated before it becomes increasingly harmful. Inaccurate classifications of patients can lead to late treatment and/or unnecessary treatment for those who don’t actually have diabetes. 

One way we will visualize results is by using a scatter plot of glucose (x-axis) vs. BMI (y-axis) where each point’s color will be linked to the outcome of whether one has type 2 diabetes or not. Using this visualization, we will be able to see the relationship between glucose and BMI and how that affects the diagnosis of diabetes. 

### Expected Outcomes and Significance

We expect to find that there is a positive correlation between glucose and BMI resulting in type 2 diabetes. Since extra glucose can be stored in the body as fat and hence BMI, this could potentially lead to higher risks of type 2 diabetes. These findings could point to the prevention of type 2 diabetes which could help aid doctors in diagnosing patients accurately. These findings could potentially lead to questions regarding the susceptibility of type 2 diabetes based on a patient's blood pressure. In addition to this, perhaps cases of type 2 diabetes in male Pima Indian heritage as well as other ethnicities could be examined. 